# Can I save the same artifacts & records twice (aka idempotency)?

The answer to this question is important both for avoiding failed workflow runs and preventing duplicated data from piling up.

## Summary

### Metadata records

LaminDB will warn you if a record with a similar `name` exists and give you a list of suggestions.

You can then decide whether you'd like to save a record to the database.

While you're free to duplicate entries with the same or similar name, in most cases, the warning will prevent that users store the same thing with different names.

### Data artifacts

The behavior for data artifacts is mostly dictated by the setting {attr}`~lamindb.dev.Settings.if_file_hash_exists`.

If you try to create an `ln.File` object from the same content,

- you'll get an existing object, if `if_file_hash_exists = "warn_return_existing"` (the default).
- you'll get an error, if `if_file_hash_exists = "error"`, 
- you'll get a warning and a new object, if `if_file_hash_exists = "warn_create_new"`.

# Examples

In [ ]:
!lamin init --storage ./test-idempotency

In [ ]:
import lamindb as ln
import pytest

ln.settings.verbosity = 1  # only show warnings

## Metadata records

Let us add a first record to the `Project` registry:

In [ ]:
project = ln.Project(name="My project 1")
project.save()

If we create a new record, we'll automatically get search results that give clues on whether we are prone to duplicating an entry:

In [ ]:
project = ln.Project(name="My project 2")

In [ ]:
project.save()

In case we match an existing name directly, we'll get the existing object:

In [ ]:
project = ln.Project(name="My project 1")

If we save it again, it will not create a new entry in the registry:

In [ ]:
project.save()

## Data artifacts

### `if_file_hash_exists = "warn_return_existing"`

In [ ]:
assert ln.settings.if_file_hash_exists == "warn_return_existing"

In [ ]:
filepath = ln.dev.datasets.file_fcs()

Create a `File` object.

In [ ]:
file = ln.File(filepath)
file.save()

In [ ]:
assert file.hash == "KCEXRahJ-Ui9Y6nksQ8z1A"

Create a `File` object from the same path:

In [ ]:
file2 = ln.File(filepath)

It gives us the existing object:

In [ ]:
assert file.id == file2.id

If you save it again, nothing will happen (the operation is idempotent):

In [ ]:
file2.save()

### `if_file_hash_exists = "error"`

In [ ]:
ln.settings.if_file_hash_exists = "error"

In this case, you'll not be able to create an object from the same content:

In [ ]:
with pytest.raises(RuntimeError):
    file3 = ln.File(filepath)

### `if_file_hash_exists = "warn_create_new"`

In [ ]:
ln.settings.if_file_hash_exists = "warn_create_new"

In this case, you'll create a new object:

In [ ]:
file4 = ln.File(filepath)
file4.save()

You can verify that it's a new entry by comparing the ids:

In [ ]:
assert file4.id != file.id

In [ ]:
file4.select(hash="KCEXRahJ-Ui9Y6nksQ8z1A").df()

In [ ]:
assert len(file.select(hash="KCEXRahJ-Ui9Y6nksQ8z1A").list()) == 2

In [ ]:
!lamin delete test-idempotency
!rm -r test-idempotency